Optimizing Inventory Management: 

**Demand Forecasting for Enhanced Inventory Control in Favorita's Expansive Grocery Retail Chain**

**Project Description:**

In today's fast-paced grocery retail sector, achieving a harmonious equilibrium between readily available products and efficient inventory costs is paramount for sustainable success. The "Optimizing Inventory Management" initiative at Favorita seeks to innovate traditional stocking methods across its numerous outlets by leveraging the potential of machine learning for precise demand forecasting.

At the heart of this project lies the ambition to rectify an operational hurdle at Favorita: ensuring consistent product availability across its diverse retail chain. The unpredictable ebb and flow of demand, influenced by seasonal shifts, changing consumer patterns, and other external factors, often results in inventory disparities. As a solution, this initiative recommends the creation and implementation of sophisticated machine learning models to accurately forecast future sales volumes.

The primary goal of this endeavor is to mine rich insights from historical sales records, comprehensive product details, pricing strategies, and promotional campaigns. Through in-depth analysis of this data, using the CRISP-DM methodology, we intend to develop machine learning algorithms adept at predicting future sales across Favorita's various outlets, thereby assisting in better inventory planning and stock replenishment strategies. This rigorous analysis and predictive approach aim not only to enhance Favorita's operational efficiency but also to ensure a seamless shopping experience for its customers.

# HYPOTHESIS

### Null Hypothesis: Promotions do not have a positive impact on overall sales.

### Alternate Hypothesis: Promotions have a positive impact on overall sales.

# Research / Analytical Questions

1. Is the train dataset complete (has all the required dates)?

2. Which dates have the lowest and highest sales for each year?

3. Did the earthquake impact sales?

4. Are certain groups of stores selling more products? (Cluster, city, state, type)

5. Are sales affected by promotions, oil prices and holidays?

6. Did the oil price affected Sales

7. How did holiday events affect Sales 

8. What analysis can we get from the date and its extractable features?

9. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

## Install the necessary modules

In [ ]:
%pip install -q pandas
%pip install -q pyodbc 
%pip install -q python-dotenv
%pip install -q matplotlib
%pip install -q seaborn
%pip install -q plotly
%pip install -q nbformat
%pip install -q statsmodels
%pip install -q dash
%pip install - qwordcloud
%pip install keras
%pip install Cython
%pip install tensorflow
%pip install keras-tuner
%pip install pmdarima
%pip install scikeras
%pip install joblib
%pip install streamlit

## Import all the necessary packages

In [ ]:
# Importing required libraries for database connectivity
import pyodbc
from dotenv import dotenv_values

# Handling warning messages
import warnings
warnings.filterwarnings('ignore')


# Libraries for data manipulation and analysis
import numpy as np
import pandas as pd

# Visualization libraries
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go 
import seaborn as sns

# Statistical libraries and functions
from scipy.stats import ttest_ind
import scipy.stats as stats
from scipy.stats import mode
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tools.sm_exceptions import ValueWarning
from sklearn.preprocessing import MinMaxScaler


# Time Series analysis libraries and functions
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Metrics and tools for model evaluation and diagnostics
import statsmodels.stats.api as sms
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Mathematical functions and utilities
from math import sqrt

# Neural network and machine learning libraries
from keras.models import Sequential
from keras.layers import Dense, LSTM
import xgboost as xgb
from sklearn.linear_model import LinearRegression

# Model selection and hyperparameter tuning libraries
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
import pmdarima as pm
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
from scipy.stats import randint
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.layers import Dropout

# to save our selected model
import joblib
import pickle

pd.set_option('display.max_row', None)

# Load Remote Dataset

In [ ]:
'''environment_variables = dotenv_values('.env')

# getting the values of the variables stored in the .env file
database = environment_variables.get('DATABASE')
server   = environment_variables.get('SERVER')
username = environment_variables.get('USERNAME')
password = environment_variables.get('PASSWORD')

# below we are creating a connection string 
#connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# below we are using the connection string to connect to the server
#connection = pyodbc.connect(connection_string)

# Now the sql query to get the data is what what you see below. 
# Note that you will not have permissions to insert delete or update this database table. 

query1 = "Select * from dbo.oil"
query2 = "Select * from dbo.stores"
query3 = "Select * from dbo.holidays_events"

# Read the sql data into dataframes
oil = pd.read_sql(query1, connection)
stores = pd.read_sql(query2, connection)
holidays_events = pd.read_sql(query3, connection)

oil.to_csv("oil.csv", index=False)
stores.to_csv("stores.csv", index=False)
holidays_events.to_csv("holidays_events.csv", index=False)'''


In [ ]:
# redefining the datatypes in the columns to be sure 

dtypes = {'id':'int64', 'store_nbr':'int8', 'onpromotion':str}

In [ ]:
# loading our datasets
data_oil = pd.read_csv("G:/AZUBI-AFRICA/CAREER_ACCELERATOR_ALL_OUT/LP3_REGR/lp3_submission_regress/lp_3_regress_sub/oil.csv", parse_dates=['date'])
df_stores = pd.read_csv("G:/AZUBI-AFRICA/CAREER_ACCELERATOR_ALL_OUT/LP3_REGR/lp3_submission_regress/lp_3_regress_sub/stores.csv")
df_holidays_events = pd.read_csv("G:/AZUBI-AFRICA/CAREER_ACCELERATOR_ALL_OUT/LP3_REGR/lp3_submission_regress/lp_3_regress_sub/holidays_events.csv", dtype={'transferred': str}, parse_dates=['date'])
df_test = pd.read_csv("G:/AZUBI-AFRICA/CAREER_ACCELERATOR_ALL_OUT/LP3_REGR/lp3_submission_regress/lp_3_regress_sub/test.csv", dtype=dtypes, parse_dates=['date'])
df_transactions = pd.read_csv("G:/AZUBI-AFRICA/CAREER_ACCELERATOR_ALL_OUT/LP3_REGR/lp3_submission_regress/lp_3_regress_sub/transactions.csv", parse_dates=['date'])

In [ ]:
df_train = pd.read_csv("G:/AZUBI-AFRICA/CAREER_ACCELERATOR_ALL_OUT/LP3_REGR/Stockholm_lp3_colla_project_sprint3/project_folder/train.csv", dtype=dtypes, parse_dates=['date'])

# Train Data

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train['family'].unique()

In [ ]:
# Find the minimum and maximum dates
min_date = df_train['date'].min()
max_date = df_train['date'].max()

print(f"The date range is from {min_date} to {max_date}")

In [ ]:
sales_time = df_train.groupby("date").sales.sum().reset_index()
sales_time = sales_time.set_index("date")

sales_time.plot(legend=False, figsize=(15, 7))
plt.title("SALES OVER TIME")
plt.ylabel("Sales")
plt.xlabel("Date")
plt.show()

### The plot indicates a rising trend in sales as the years progress.

In [ ]:
df_tr = df_train.copy()
# Set 'date' as the index
df_tr.set_index('date', inplace=True)

# Resample and calculate the mean
daily_sales = df_tr['sales'].resample("D").mean()

# Plot
plt.figure(figsize=(15,7))
daily_sales.plot(title='Average Daily Sales')
plt.ylabel('Average Sales')
plt.xlabel('Date')
plt.tight_layout()
plt.show()

In [ ]:
# Resample and calculate the mean
daily_sales = df_tr['sales'].resample('W').mean()

# Plot
plt.figure(figsize=(15,7))
daily_sales.plot(title='Average Weekly Sales')
plt.ylabel('Average Sales')
plt.xlabel('Date')
plt.tight_layout()
plt.show()

In [ ]:
# Resample and calculate the mean
daily_sales = df_tr['sales'].resample('M').mean()

# Plot
plt.figure(figsize=(15,7))
daily_sales.plot(title='Average Monthly Sales')
plt.ylabel('Average Sales')
plt.xlabel('Date')
plt.tight_layout()
plt.show()

# Stores Data

In [ ]:
df_stores.head()

In [ ]:
df_stores.shape

In [ ]:
df_stores.info()

In [ ]:
df_stores.describe()

In [ ]:
df_stores.isnull().sum()

In [ ]:
# Compute counts of each store type
stores = df_stores['type'].value_counts()

# Visualization
plt.figure(figsize=(15, 5))

sns.barplot(x=stores.values, y=stores.index)
plt.title('Counts of Store Types')
plt.ylabel('Store Types')
plt.xlabel("Number of Stores")
plt.show()

### Store type D has the highest number of stores, while type E has the lowest number of stores

In [ ]:
plt.figure(figsize=(15, 5))
sns.countplot(data=df_stores, x='city')
plt.xticks(rotation=90)
plt.title('Number of Stores by City')
plt.show()

### Quito has the highest number of stores

In [ ]:
# Number of Stores by State
plt.figure(figsize=(15, 5))

# Order the states based on their counts (from highest to lowest)
order = df_stores['state'].value_counts().index

sns.countplot(data=df_stores, x='state', order=order)
plt.xticks(rotation=90)
plt.title('Number of Stores by State')
plt.show()

### Pichincha has the highest number of stores

In [ ]:
# Distribution of clusters
plt.figure(figsize=(15, 5))

# Order the clusters based on their counts (from highest to lowest)
order = df_stores['cluster'].value_counts().index

sns.countplot(data=df_stores, x='cluster', order=order)
plt.title('Distribution of Clusters')
plt.show()

### Cluster 3 has the highest number of stores in it

# Oil Data

In [ ]:
data_oil.head()

In [ ]:
data_oil.shape

In [ ]:
data_oil.info()

In [ ]:
data_oil.describe()

In [ ]:
data_oil.isnull().sum()

In [ ]:
total_rows = len(data_oil)
null_rows = data_oil['dcoilwtico'].isnull().sum()

percentage_null = (null_rows / total_rows) * 100
print(f"Percentage of null values in 'dcoilwtico' column: {percentage_null:.2f}%")

In [ ]:
# Use linear interpolation to impute missing values
data_oil['dcoilwtico'] = data_oil['dcoilwtico'].interpolate(method='linear')

In [ ]:
# Find the minimum and maximum dates
min_date = data_oil['date'].min()
max_date = data_oil['date'].max()

print(f"The date range is from {min_date} to {max_date}")

In [ ]:
# Creating a line plot using plotly.express
fig = px.line(data_oil, x='date', y='dcoilwtico', title='Oil Prices Over Time')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Oil Price')
fig.show()

This shows a downward trend in the prices of oil prices on the market over time.

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=data_oil, x='dcoilwtico', bins=20, kde=True)
plt.title("Distribution of Oil Prices")
plt.xlabel("Oil Price (dcoilwtico)")
plt.ylabel("Frequency")
plt.show()

The graph shows that the price of crude oil was heavily distributed between 40 and $60 a barrel, per day. Most of the prices of crude oil per day were between this price range.

# Holiday Data

In [ ]:
df_holidays_events.head() 

In [ ]:
df_holidays_events.drop(columns=['description'], inplace=True)

In [ ]:
df_holidays_events.shape

In [ ]:
df_holidays_events.describe(include='object')

In [ ]:
df_holidays_events.info()

In [ ]:
df_holidays_events.isnull().sum()

In [ ]:
# Find the minimum and maximum dates
min_date = df_holidays_events['date'].min()
max_date = df_holidays_events['date'].max()

print(f"The date range is from {min_date} to {max_date}")

In [ ]:
# Visualizing holiday count by different types
categorical_columns = ['type', 'locale', 'locale_name', 'transferred']

for column in categorical_columns:
    plt.figure(figsize=(15,7))
    sns.countplot(data=df_holidays_events, x=column)
    plt.title(f'Count of Holidays by {column}')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.xticks(rotation=45)  
    plt.show()

### Ecuador had the highest number of holidays
### National holidays had the highest count according to locale
### Very few holidays were transferred

In [ ]:
plt.figure(figsize=(10, 6))
df_holidays_events.groupby(df_holidays_events['date'].dt.year)['date'].count().plot(kind='line')
plt.title('Distribution of Holidays Over Years')
plt.xlabel('Year')
plt.ylabel('Count')
plt.show()

### From 2012 to 2014, we observed a consistent upward trend in the number of holidays. However, there was a noticeable dip in 2015. Interestingly, the number of holidays surged in 2016 but then followed by a decline in 2017.

In [ ]:
transferred_counts = df_holidays_events[df_holidays_events['type'] == 'Transfer'].groupby(df_holidays_events['date'].dt.year)['type'].count()

# Plot the count of transferred holidays by year
plt.figure(figsize=(10, 6))
transferred_counts.plot(kind='bar')
plt.title('Count of Transferred Holidays by Year')
plt.xlabel('Year')
plt.ylabel('Count')
plt.show()

### 2017 had the most transferred holidays

In [ ]:
# Plotting distribution of event holidays on yearly basis

event_data = df_holidays_events[df_holidays_events['type'] == 'Event']

plt.figure(figsize=(12, 6))
event_data.groupby(event_data['date'].dt.year).size().plot(kind='bar')
plt.title('Distribution of Event Holidays on Yearly Basis')
plt.xlabel('Year')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

### 2016 had the most holidays

# Transaction Data

In [ ]:
df_transactions.head()

In [ ]:
df_transactions.shape

In [ ]:
df_transactions.describe()

In [ ]:
df_transactions.info()

In [ ]:
df_transactions.isnull().sum()

In [ ]:
# Find the minimum and maximum dates
min_date = df_transactions['date'].min()
max_date = df_transactions['date'].max()

print(f"The date range is from {min_date} to {max_date}")

In [ ]:
transactions_date = df_transactions.groupby('date').transactions.sum().reset_index()
transactions_date = transactions_date.set_index("date")

transactions_date.plot(legend=False, color="skyblue", figsize=(15, 7))
plt.title('Number of Transactions Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Transactions')
plt.style.use("fivethirtyeight")
plt.show()

### Notably, transaction volumes surge towards the end of each year and then exhibit a decline at the onset of the subsequent year.

In [ ]:
# resample the transaction data to month
trans_res= df_transactions.set_index('date').drop('store_nbr', axis=1).resample('M').mean()
plt.figure(figsize=(15,6))
trans_res.plot()
plt.title('Average Monthly Transactions')

In [ ]:
# Group by store and sum the transactions
store_transactions = df_transactions.groupby('store_nbr')['transactions'].sum().reset_index()

# Sort the transactions in descending order
sorted_stores = store_transactions.sort_values(by='transactions', ascending=False)

# Take top 10 stores for visualization
top_10_stores = sorted_stores.head(10)

# Visual representation of top 10 stores
plt.figure(figsize=(15,7))
sns.barplot(x='store_nbr', y='transactions', data=top_10_stores, order=top_10_stores['store_nbr'])
plt.title("Top 10 Stores by Transactions")
plt.xlabel("Store Number")
plt.ylabel("Total Transactions")
plt.xticks(rotation=45)  # Rotate x labels for better visibility
plt.tight_layout()  # Adjust layout for better visibility
plt.show()

### Store Number 44 had the highest number of transactions

# Test Data

In [ ]:
df_test.head()

In [ ]:
df_test.shape

In [ ]:
df_test.info()

In [ ]:
df_test.describe()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test['family'].unique()

# Research / Analytical Questions

Is the train dataset complete (has all the required dates)?

Which dates have the lowest and highest sales for each year?

Did the earthquake impact sales?

Are certain groups of stores selling more products? (Cluster, city, state, type)

Are sales affected by promotions, oil prices and holidays?

Did the oil price affected Sales

How did holiday events affect Sales 

What analysis can we get from the date and its extractable features?

What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

1. Is the train dataset complete (has all the required dates)?

In [ ]:
# Get the minimum and maximum dates in the DataFrame
min_date = df_train['date'].min()
max_date = df_train['date'].max()

# Create a date range for the entire period
all_dates = pd.date_range(start=min_date, end=max_date)

# Check if there are any dates missing in the DataFrame
missing_dates = all_dates.difference(df_train['date'])

if missing_dates.empty:
    print("The train dataset is complete and has all the required dates.")
else:
    print(f"The train dataset is missing the following dates:\n{missing_dates}")


2.Which dates have the lowest and highest sales for each year?

In [ ]:
# Extract year from date and add it as a new column
df_train['year'] = df_train['date'].dt.year

# Group by year and find the date with max and min sales
max_sales_dates = df_train.loc[df_train.groupby('year')['sales'].idxmax()][['year', 'date', 'sales']]
min_sales_dates = df_train.loc[df_train.groupby('year')['sales'].idxmin()][['year', 'date', 'sales']]

# Adding a column to distinguish between max and min sales
max_sales_dates['Type'] = 'Max Sales'
min_sales_dates['Type'] = 'Min Sales'

# Concatenating both dataframes and sorting by sales in descending order 
# (and then by year and Type for a hierarchical sort)
result_df = pd.concat([max_sales_dates, min_sales_dates], ignore_index=True)
result_df = result_df.sort_values(by=['sales', 'year', 'Type'], ascending=[False, True, False])

result_df

In [ ]:
# Visualization
plt.figure(figsize=(10,6))

for _, row in max_sales_dates.iterrows():
    plt.plot(row['date'], row['sales'], 'go', label=f"Max {row['year']}")

for _, row in min_sales_dates.iterrows():
    plt.plot(row['date'], row['sales'], 'ro', label=f"Min {row['year']}")

plt.plot(df_train['date'], df_train['sales'], label='Sales', alpha=0.5)
plt.title("Highest and Lowest Sales Dates for Each Year")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

3. Did the earthquake impact sales?

In [ ]:
# Filter data for the year 2016
df_2016 = df_train[df_train['date'].dt.year == 2016].set_index('date')

# Resample data to weekly frequency
weekly_sales = df_2016.resample('W').sum()

# Date of the earthquake
earthquake_date = "2016-04-16"

# Visualization for weekly sales
plt.figure(figsize=(12,6))
sns.lineplot(data=weekly_sales, x=weekly_sales.index, y='sales', label='Weekly Sales', color='blue')
plt.axvline(pd.to_datetime(earthquake_date), color='green', linestyle='--', label='Earthquake Date')
plt.title("Weekly Sales in 2016")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Resample data to monthly frequency
monthly_sales = df_2016.resample('M').sum()

# Visualization for monthly sales
plt.figure(figsize=(12,6))
sns.lineplot(data=monthly_sales, x=monthly_sales.index, y='sales', label='Monthly Sales', color='blue')
plt.axvline(pd.to_datetime(earthquake_date), color='green', linestyle='--', label='Earthquake Date')
plt.title("Monthly Sales in 2016")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
earthquake_date = "2016-04-16"

# Splitting data into pre and post earthquake
pre_earthquake = df_2016[df_2016.index < pd.to_datetime(earthquake_date)]
post_earthquake = df_2016[df_2016.index >= pd.to_datetime(earthquake_date)]

# Extract sales data for both periods
pre_sales = pre_earthquake['sales']
post_sales = post_earthquake['sales']

# Perform t-test
t_stat, p_value = ttest_ind(pre_sales, post_sales, equal_var=True)

print(f"t-statistic: {t_stat}")
print(f"p-value: {p_value}")

# Check if p-value is less than the common significance level of 0.05
if p_value < 0.05:
    print("The earthquake had a statistically significant impact on sales.")
else:
    print("The earthquake did not have a statistically significant impact on sales.")


### The earthquake had a significant impact on the sales.

4. Are certain groups of stores selling more products? (Cluster, city, state, type)

In [ ]:
df_stores.head()

In [ ]:
# Merging the data
df_merged = df_stores.merge(df_train, on='store_nbr')

# Visualizing sales by each group
for group in ['cluster', 'type']:
    plt.figure(figsize=(10, 6))
    aggregated_data = df_merged.groupby(group)['sales'].sum().sort_values()
    sns.barplot(x=aggregated_data.index, y=aggregated_data.values, palette="viridis")
    
    plt.title(f"Total Sales by {group}")
    plt.ylabel('Total Sales')
    plt.xlabel(group)
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()
    
# Visualizing sales by city and state with horizontal bar plots
for group in ['city', 'state']:
    plt.figure(figsize=(10, 6))
    aggregated_data = df_merged.groupby(group)['sales'].sum().sort_values()
    sns.barplot(y=aggregated_data.index, x=aggregated_data.values, palette="viridis", orient='h')
    
    plt.title(f"Total Sales by {group}")
    plt.xlabel('Total Sales')
    plt.ylabel(group.capitalize())
    plt.grid(axis='x')
    plt.tight_layout()
    plt.show()

### Cluster 14 had the highest sales
### Store types D & A had the highest sales
### Stores in Quito had the highest sales
### Stores in Pichincha had the highest sales

5. Are sales affected by promotions, oil prices and holidays?

In [ ]:
# Merge sales and oil prices on 'date'
df_merged = pd.merge(df_train, data_oil, on='date', how='left')

# Merge the result with holidays on 'date'
df_merged = pd.merge(df_merged, df_holidays_events, on='date', how='left')

# Fill NaN values in the 'type', 'locale', 'locale_name', 'description', 'transferred' columns with 'NoHoliday' or appropriate placeholder
df_merged[['type', 'locale', 'locale_name']] = df_merged[['type', 'locale', 'locale_name']].fillna('NoHoliday')
df_merged['transferred'] = df_merged['transferred'].fillna(False)

df_merged.head()


6. Did the oil price affected Sales

In [ ]:
# Grouping the data by oil price and then calculating the total sales
oil_price_sales = df_merged.groupby('dcoilwtico')['sales'].sum()

# Creating a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=oil_price_sales.index, y=oil_price_sales.values)
plt.title('Total Sales vs Oil Price')
plt.xlabel('Oil Price (dcoilwtico)')
plt.ylabel('Total Sales')
plt.show()

Higher Sales at Lower Oil Prices: There's a dense concentration of sales data points when the oil price is on the lower end. This indicates that when oil prices were lower, sales were generally higher.

Sales Dwindle with Rising Oil Prices: As oil prices increase, there's a noticeable reduction in the concentration of sales. This suggests that higher oil prices might have a dampening effect on sales.

In summary, there's a trend suggesting that the oil price significantly impacts sales. Lower oil prices are associated with higher sales, whereas higher oil prices correspond with reduced sales concentrations.

7. How did holiday events affect Sales

In [ ]:
# Create a list to hold sales data for each holiday type
data_groups = [df_merged['sales'][df_merged['type'] == holiday_type] for holiday_type in df_merged['type'].unique()]

# Perform ANOVA
f_value, p_value = stats.f_oneway(*data_groups)

print(f"F-value: {f_value}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print("The different holiday types have a statistically significant impact on sales.")
else:
    print("The different holiday types do not have a statistically significant impact on sales.")


In [ ]:
# Grouping the data by holiday_type and then calculating average sales
holiday_sales = df_merged.groupby('type')['sales'].mean().sort_values(ascending=False)

# Creating a bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x=holiday_sales.index, y=holiday_sales.values, palette="viridis")
plt.title('Average Sales during Holiday Types')
plt.xlabel('Holiday Type')
plt.ylabel('Average Sales')
plt.xticks(rotation=45)
plt.show()


8. What analysis can we get from the date and its extractable features?

In [ ]:
df_train_copy = df_train.copy()

# extract year, quarter, month, day, and weekday information from the date column
df_train_copy['year'] = df_train_copy['date'].dt.year
df_train_copy['month'] = df_train_copy['date'].dt.month
df_train_copy['week'] = df_train_copy['date'].dt.isocalendar().week
df_train_copy['day'] = df_train_copy['date'].dt.day
df_train_copy['weekday'] = df_train_copy['date'].dt.weekday

# plot the aggregated sales data by year
grouped_by_year = df_train_copy.groupby('year')['sales'].sum()
plt.figure(figsize=(15, 7))
plt.plot(grouped_by_year.index, grouped_by_year.values)
plt.xlabel("Year")
plt.ylabel("Sales")
plt.title("Sales by Year")
plt.show()

# plot the aggregated sales data by month
grouped_by_month = df_train_copy.groupby('month')['sales'].sum()
plt.figure(figsize=(15, 7))
plt.bar(grouped_by_month.index, grouped_by_month.values)
plt.xlabel("Month")
plt.ylabel("Sales")
plt.title("Sales by Month")
plt.show()

# plot the aggregated sales data by week
grouped_by_week = df_train_copy.groupby('week')['sales'].sum()
plt.figure(figsize=(15, 7))
plt.plot(grouped_by_week.index, grouped_by_week.values)
plt.xlabel("Week")
plt.ylabel("Sales")
plt.title("Sales by Week")
plt.show()

# plot the aggregated sales data by day
grouped_by_day = df_train_copy.groupby('day')['sales'].sum()
plt.figure(figsize=(15, 7))
plt.plot(grouped_by_day.index, grouped_by_day.values)
plt.xlabel("Day")
plt.ylabel("Sales")
plt.title("Sales by Day")
plt.show()

# plot the aggregated sales data by weekday
grouped_by_weekday = df_train_copy.groupby('weekday')['sales'].sum()
plt.figure(figsize=(15, 7))
plt.plot(grouped_by_weekday.index, grouped_by_weekday.values)
plt.xlabel("Weekday")
plt.ylabel("Sales")
plt.title("Sales by Weekday")
plt.show()

## Time Series Decomposition of Sales Data

The given code aims to decompose the sales data over various time frames and granularities. By breaking down the data based on year, month, week, day, and weekday, it provides a comprehensive view of sales patterns and trends. Here's a breakdown of the code:

- **Year**: The year of the transaction.
- **Month**: The month of the transaction.
- **Week**: The week of the year the transaction occurred.
- **Day**: The day of the month.
- **Weekday**: The day of the week (0 denotes Monday and 6 represents Sunday).

### Visualizing Sales Trends

1. **Sales by Year**: A line plot showing yearly aggregated sales provides insights into the yearly sales trend and highlights any consistent growth or decline.

2. **Sales by Month**: A bar plot showcasing monthly aggregated sales gives an understanding of the monthly sales distribution, potentially uncovering seasonality patterns or specific months with higher sales.

3. **Sales by Week**: The line plot of weekly aggregated sales lets us see if there are specific weeks in a year where sales peak or dip.

4. **Sales by Day**: By plotting daily aggregated sales, we can determine if certain days of the month consistently perform better in terms of sales.

5. **Sales by Weekday**: This line plot offers insights into sales performance by weekdays, which can be helpful in understanding weekly patterns, like whether sales are higher on weekends or weekdays.

In essence, this comprehensive analysis allows businesses to understand their sales patterns better, pinpointing periods of peak sales and understanding potential cyclic behaviors in their sales data.

7. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

1. **Mean Absolute Error (MAE)**: This metric calculates the average of absolute differences between the forecasted values and the actual values. It provides a linear penalty for each unit of difference. 

    \[ \text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i| \]

2. **Mean Squared Error (MSE)**: This metric calculates the average of the squares of the differences between the forecasted values and the actual values. It provides a quadratic penalty for each unit of difference, which means larger errors are more heavily penalized.

    \[ \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 \]

3. **Root Mean Squared Error (RMSE)**: This is simply the square root of MSE. It's a popular metric because it represents the sample standard deviation of the differences between predicted values and observed values.

    \[ \text{RMSE} = \sqrt{\text{MSE}} \]

4. **Root Mean Squared Logarithmic Error (RMSLE)**: It calculates the difference between the log-transformed predicted and actual values. It's particularly useful when you care more about percentage errors rather than the absolute value of errors.

    \[ \text{RMSLE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (\log(y_i + 1) - \log(\hat{y}_i + 1))^2} \]

**Why might MAE be greater than RMSE?**

It's not necessarily true that MAE is always greater than RMSE, but it can be, depending on the dataset. Since RMSE squares the errors before averaging and then takes the square root, it tends to give more weight to larger errors. This means RMSE should be more useful when large errors are particularly undesirable. Here are some differences:

- **Outliers**: RMSE is highly sensitive to outliers, whereas MAE is robust to outliers. If you have many outliers, the squared differences can lead to a much higher RMSE compared to MAE.
  
- **Error Distribution**: If all errors are consistently of the same magnitude, then MAE and RMSE will be similar. However, if some errors are much larger than others (a mix of small and large errors), RMSE will tend to be larger than MAE.

- **Interpretability**: MAE directly represents the average error magnitude, which can sometimes be more interpretable than RMSE.

In general, if you're comparing two models, it's a good practice to consider both MAE and RMSE as they can provide different insights into the errors the models are making.

# HYPOTHESIS

### Null Hypothesis: Promotions do not have a positive impact on overall sales.

### Alternate Hypothesis: Promotions have a positive impact on overall sales.

In [ ]:
# Extracting the 'sales' column for each group
sales_promotion = df_merged[df_merged['onpromotion'] == 1]['sales']
sales_no_promotion = df_merged[df_merged['onpromotion'] == 0]['sales']

# Performing a two-sample t-test
t_statistic, p_value = stats.ttest_ind(sales_promotion, sales_no_promotion, equal_var=False)

# Defining the significance level (alpha)
alpha = 0.05

# Comparing the p-value to the significance level
if p_value < alpha:
    print("Reject the null hypothesis.")
    print("Promotions have a significant impact on sales.")
else:
    print("Fail to reject the null hypothesis.")
    print("There is no significant impact of promotions on sales.")

Based on the results of the hypothesis test, it appears that there is no significant impact of promotions on sales. This means that there is insufficient evidence to conclude that promotions lead to a statistically significant increase in sales.

# Merge the Datasets

In [ ]:
# Merge df_stores and df_train on 'store_nbr'
df_merged = df_train.merge(df_stores, on='store_nbr', how='left')

# Merge with data_oil on 'date'
df_merged = df_merged.merge(data_oil, on='date', how='left')

# Merge the result with df_holidays_events on 'date'
df_merged = df_merged.merge(df_holidays_events, on='date', how='left')

# Merge with df_transactions on 'date' and 'store_nbr'
#df_merged = df_merged.merge(df_transactions, on=['date', 'store_nbr'], how='left')

# Fill NaN values in the specified columns
df_merged[['type_y', 'locale', 'locale_name']] = df_merged[['type_y', 'locale', 'locale_name']].fillna('NoHoliday')
df_merged['transferred'] = df_merged['transferred'].fillna(False)

df_merged.head()


In [ ]:
#Interpolating the oil prices
df_merged['dcoilwtico'].fillna(method='ffill', inplace=True)  # Forward fill
df_merged['dcoilwtico'].fillna(method='bfill', inplace=True)  # Backfill any remaining NaNs

In [ ]:
df_merged['onpromotion'] = pd.to_numeric(df_merged['onpromotion'], errors='coerce')

In [ ]:
numeric_df = df_merged.select_dtypes(include=['float64', 'int64'])
corr_matrix = numeric_df.corr()


In [ ]:
# Set 'id' column as the index
df_merged.set_index('id', inplace=True)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Columns to drop
columns_to_drop = [ 'city', 'state', 'type_x', 'dcoilwtico', 
                   'type_y', 'locale', 'locale_name', 'transferred']
# Drop the columns
df_merged = df_merged.drop(columns=columns_to_drop)



In [ ]:
# Rearrange columns to make 'sales' the last column
cols = [col for col in df_merged.columns if col != 'sales'] + ['sales']
df_merged = df_merged[cols]

In [ ]:
# Aggregating data #'family'

df_family_aggregated = df_merged.groupby(['date', 'family']).agg({
    'sales': 'sum',
    'onpromotion': 'sum'  # This can be a mean (indicating the proportion of items on promotion) or sum.    
}).reset_index()


why are we aggregating you ask?

When testing properties such as presence of a unit root, what matters is the time span, not the frequency at which the data is sampled. Unless the sample is too small for asymptotic inference, that is.

http://davegiles.blogspot.no/2014/05/unit-root-testing-sample-size-vs-sample.html
http://davegiles.blogspot.no/2014/09/the-econometrics-of-temporal.html#more
https://johnhcochrane.blogspot.com/2015/04/unit-roots-redux.html#more



In [ ]:
df_family_aggregated.isnull().sum()

In [ ]:
print(df_family_aggregated.dtypes)




<h1 style='background:#0A4D68; border:3; color:cyan; border-color:cyan; border-style:dotted;'><center> Feature Engineering</center></h1> 


Sales can vary significantly depending on the day of the week. For example, retailers might observe more sales on weekends as compared to weekdays. By including the day of the week as a predictor, the model can capture such weekly patterns.

Lagged variables help the model understand temporal dependencies. If sales today depend on sales from the previous day or the same day from the previous week, these lagged variables can capture that relationship.

Rolling statistics can smooth out short-term fluctuations and highlight longer-term trends or cycles in the data. This helps in capturing seasonality or patterns that recur over fixed periods.

To improve the prediction of sales, the dataset has been enhanced by:

Incorporating the day of the week to account for weekly sales patterns.

Adding lagged sales variables to capture short-term dependencies and potential weekly patterns.

Introducing rolling statistics, specifically a 7-day rolling mean, to smooth out short-term variations and emphasize longer-term trends.

In [ ]:
# 1. Day of the week (0 = Monday, 6 = Sunday)
df_family_aggregated['day_of_week'] = df_family_aggregated['date'].dt.dayofweek

# 2. Lagged variables
# Here I'm creating a 1-day lag for sales. 
df_family_aggregated['lag_1'] = df_family_aggregated['sales'].shift(1)

# df_family_aggregated['lag_7'] = df_family_aggregated['sales'].shift(7)  # 1-week lag

# 3. Rolling statistics
window_size = 7  # 7-day window, you can adjust this based on your data's frequency and need
df_family_aggregated['rolling_mean'] = df_family_aggregated['sales'].rolling(window=window_size).mean()

In [ ]:
df_family_aggregated.isnull().sum()

In [ ]:
# Drop the rows with NaN values that were introduced due to lag and rolling stats calculations
df_family_aggregated.dropna(inplace=True)

In [ ]:
print(df_family_aggregated.dtypes)

## Checking if the new Variables added are statistically significant in predicting Sales

In [ ]:
# Defining our independent variables (X) and dependent variable (y)
X = df_family_aggregated[['onpromotion', 'day_of_week', 'lag_1', 'rolling_mean']]
X = sm.add_constant(X)  # Adds a constant term to the predictor
y = df_family_aggregated['sales']

# Fit the regression model
model = sm.OLS(y, X).fit()

# Display the regression results
print(model.summary())

const: The p-value is practically 0 (significantly less than 0.05). This means that the intercept (const) is statistically significant, and its estimated value is different from zero in the population.

onpromotion: The p-value is also essentially 0. This indicates that the coefficient for the onpromotion variable is statistically significant, implying that promotions have a significant effect on sales.

day_of_week: The p-value here is less than 0.05, suggesting that the day_of_week coefficient is statistically significant. Therefore, the day of the week has a significant influence on sales.

lag_1: With a p-value that's virtually 0, the lag_1 coefficient is statistically significant. This implies that the sales value of the previous day (or period) significantly affects the current sales.

rolling_mean: The p-value for this variable is again essentially 0. It suggests that the rolling mean is a statistically significant predictor for sales in the model.

## Performing the VIF test to check for multicollinearity

In [ ]:
# Check VIF scores for multicollinearity
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

VIF = 1: No multicollinearity.

1 < VIF < 5: Moderately correlated. In many fields, a VIF below 5 is considered acceptable.

VIF >= 5: Highly correlated. A VIF above 5 is typically considered to indicate high multicollinearity.

const: VIF of 4.142 suggests that the intercept is moderately correlated with other predictors. This is not typically a concern since it's the constant.

onpromotion: A VIF of 1.1527 indicates little to no multicollinearity.

day_of_week: With a VIF close to 1 (1.0181), this predictor is not showing multicollinearity.

lag_1: A VIF of 1.1991 also suggests very minimal multicollinearity.

rolling_mean: A VIF of 1.3610 indicates little multicollinearity but slightly more than the other predictors (excluding the constant).




<h1 style='background:#0A4D68; border:3; color:cyan; border-color:cyan; border-style:dotted;'><center> Visualizing the Sales Column</center></h1> 


In [ ]:
# 1. Visualization
plt.figure(figsize=(12, 6))
df_family_aggregated['sales'].plot(title='Total Daily Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()

The Autocorrelation Function (ACF) is a tool that provides information about the linear relationship between values of a time series separated by various time lags. In simpler terms, it tells you how correlated a time series is with itself at different lagged values.
A stationary time series will have an ACF that drops to zero relatively quickly, while the ACF of non-stationary data decreases slowly. Also, for a non-stationary series, the ACF will often have values that remain significant over several lags, which is indicative of a strong time-dependent structure.

## Plotting the ACF

In [ ]:
# ACF Plot
plot_acf(df_family_aggregated['sales'], lags=40)
plt.title('Autocorrelation Function')
plt.show()

A stationary time series will have an ACF that decays to zero relatively quickly. This means that the correlation between the current value of the time series and its past values decreases as the lag increases. The ACF decays to zero relatively quickly meaning it is stationary but we have to do a formal test to confirm this 




<h1 style='background:#0A4D68; border:3; color:cyan; border-color:cyan; border-style:dotted;'><center> Augmented Dickey-Fuller (ADF)</center></h1> 

The Augmented Dickey-Fuller (ADF) test is a formal statistical test for stationarity. Specifically, it's used to determine the presence of a unit root in a time series, which would suggest that the time series is non-stationary. 

In [ ]:
# ADF Test
result = adfuller(df_family_aggregated['sales'])
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])
if result[1] <= 0.05:
    print("The time series is stationary.")
else:
    print("The time series is not stationary.")  


<h1 style='background:#0A4D68; border:3; color:cyan; border-color:cyan; border-style:dotted;'><center> DATA PREPROCESSING</center></h1> 

In [ ]:
# One-hot encoding for family column
df_family_dummies = pd.get_dummies(df_family_aggregated['family'], drop_first=True, prefix='family')
df_family_aggregated = pd.concat([df_family_aggregated, df_family_dummies], axis=1)
df_family_aggregated.drop('family', axis=1, inplace=True)

In [ ]:
# Features to scale
columns_to_scale = ['onpromotion', 'day_of_week', 'lag_1', 'rolling_mean', 'sales'] + list(df_family_dummies.columns)#'is_holiday'

scaler = MinMaxScaler()
df_family_aggregated[columns_to_scale] = scaler.fit_transform(df_family_aggregated[columns_to_scale])

In [ ]:
df_family_aggregated = df_family_aggregated.sort_values('date')

In [ ]:
train_size = int(0.8 * len(df_family_aggregated))
train_data = df_family_aggregated[:train_size]
test_data = df_family_aggregated[train_size:]

X_train = train_data.drop(columns=['sales', 'date'])
y_train = train_data['sales']
X_test = test_data.drop(columns=['sales', 'date'])
y_test = test_data['sales']

In [141]:
X_train.head()

,onpromotion,day_of_week,lag_1,rolling_mean,family_BABY CARE,family_BEAUTY,family_BEVERAGES,family_BOOKS,family_BREAD/BAKERY,family_CELEBRATION,...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
6,0.0,0.166667,0.000190,0.000642,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.166667,0.000000,0.000763,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.166667,0.000195,0.000856,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.166667,0.000150,0.000900,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.166667,0.000075,0.000405,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
# below are saving the train set as a csv file so we can load when creating our streamlit app

X_train.to_csv('X_train.csv', index=False)


<h1 style='background:#0A4D68; border:3; color:cyan; border-color:cyan; border-style:dotted;'><center> Model Selection</center></h1> 

ARIMA: Effective for data with no seasonality. Works well when a series is stationary or can be made stationary through differencing (hence the "Integrated" part in ARIMA).

SARIMA: An extension of ARIMA to handle seasonality in the data.

Exponential Smoothing: Useful for data with a trend or seasonality. There are variations like Holt (for trend) and Holt-Winters (for both trend and seasonality).

LSTM: A type of recurrent neural network. Can capture long-term dependencies and complex patterns in the data. Typically requires more data and more computational resource

XGB Boost


<h1 style='background:#0A4D68; border:3; color:cyan; border-color:cyan; border-style:dotted;'><center> ACF AND PACF</center></h1> 

The Autocorrelation Function (ACF) and the Partial Autocorrelation Function (PACF) plots are fundamental diagnostic tools. They help us understand the underlying characteristics of a time series dataset, and more specifically, they are typically used when modeling the data using Autoregressive Integrated Moving Average (ARIMA) models or its variants.

In [ ]:
# Plot ACF
plt.figure(figsize=(10, 5))
plot_acf(df_family_aggregated['sales'], lags=80)  # you can adjust the lags as per your data's requirements
plt.title('ACF Plot')
plt.show()

In [ ]:
# Plot PACF
plt.figure(figsize=(10, 5))
plot_pacf(df_family_aggregated['sales'], lags=40)  # again, adjust lags as necessary
plt.title('PACF Plot')
plt.show()

The appropriate ARIMA model for our dataset is ARIMA(1,0,0). This is because only the first lag exhibits statistical significance, suggesting an autoregressive order of 1. Additionally, there's no indication of a moving average component, and differencing of the target variable was not required.


<h1 style='background:#0A4D68; border:3; color:cyan; border-color:cyan; border-style:dotted;'><center> FITTING MODELS</center></h1> 

In [ ]:
# storing the exogenous variables

exog_features = ['onpromotion', 'day_of_week', 'lag_1', 'rolling_mean']

In [ ]:
# RMSLE Definition
def rmsle(predicted, actual):
    return np.sqrt(np.mean(np.square(np.log1p(predicted) - np.log1p(actual))))

In [ ]:
# Suppress the ValueWarning warnings
warnings.simplefilter(action='ignore', category=ValueWarning)

In [ ]:
# Fitting Arima Model
model_arima = sm.tsa.SARIMAX(y_train, exog=X_train[exog_features], order=(1,0,0))
results_arima = model_arima.fit(disp=False)
forecast_arima = results_arima.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1, exog=X_test[exog_features])

# Visualization
plt.figure(figsize=(10,6))
plt.plot(test_data['date'], y_test, label='Actual')
plt.plot(test_data['date'], forecast_arima, color='red', label='ARIMA Forecast')
plt.legend()
plt.title('ARIMA Forecast vs Actuals')
plt.show()

mse_arima = mean_squared_error(y_test, forecast_arima)
rmse_arima = np.sqrt(mse_arima)
rmsle_arima = rmsle(forecast_arima, y_test)

In [ ]:
# Fitting sarima Model
model_sarima = sm.tsa.SARIMAX(y_train, exog=X_train[exog_features], order=(1,1,0), seasonal_order=(1,0,0,12))
results_sarima = model_sarima.fit(disp=False)
forecast_sarima = results_sarima.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1, exog=X_test[exog_features])

#Visualization
plt.figure(figsize=(10,6))
plt.plot(test_data['date'], y_test, label='Actual')
plt.plot(test_data['date'], forecast_sarima, color='red', label='SARIMA Forecast')
plt.legend()
plt.title('SARIMA Forecast vs Actuals')
plt.show()

mse_sarima = mean_squared_error(y_test, forecast_sarima)
rmse_sarima = np.sqrt(mse_sarima)
rmsle_sarima = rmsle(forecast_sarima, y_test)

In [ ]:
# 1. Fit ETS Model and Make Initial Forecasts
model_ets = ExponentialSmoothing(y_train, seasonal='add', seasonal_periods=12)
results_ets = model_ets.fit()
initial_forecast = results_ets.forecast(steps=len(y_test))

# 2. Prepare Adjusted Exogenous Data for Linear Regression

# Using the initial forecast and exog features to predict y_test.
X_adjust_train = pd.concat([y_train.shift(-1).dropna().rename("forecast"), X_train[exog_features].iloc[1:, :]], axis=1)
X_adjust_test = pd.concat([pd.Series(initial_forecast, index=y_test.index).rename("forecast"), X_test[exog_features]], axis=1)

# Remove rows with NaN values
X_adjust_train = X_adjust_train.dropna()
y_adjust_train = y_train.loc[X_adjust_train.index]

X_adjust_test = X_adjust_test.dropna()
y_adjust_test = y_test.loc[X_adjust_test.index]

# Convert all feature names to strings
X_adjust_train.columns = X_adjust_train.columns.astype(str)
X_adjust_test.columns = X_adjust_test.columns.astype(str)

# 3. Fit the Linear Regression Model

lr = LinearRegression()
lr.fit(X_adjust_train, y_adjust_train)
forecast_ets = lr.predict(X_adjust_test)

# 4. Visualization
plt.figure(figsize=(10,6))
plt.plot(y_adjust_test.index, y_adjust_test, label='Actual')
plt.plot(y_adjust_test.index, forecast_ets, color='red', label='Adjusted ETS Forecast')
plt.legend()
plt.title('Adjusted ETS Forecast vs Actuals')
plt.show()

# 5. Performance Metrics
mse_ets = mean_squared_error(y_adjust_test, forecast_ets)
rmse_ets = np.sqrt(mse_ets)
rmsle_ets = rmsle(forecast_ets, y_adjust_test)

In [ ]:
#Fitting XGB Boost Regressor
model_xgb = xgb.XGBRegressor(objective ='reg:squarederror')
model_xgb.fit(X_train, y_train)
forecast_xgb = model_xgb.predict(X_test)

# Visualization
plt.figure(figsize=(10,6))
plt.plot(test_data['date'], y_test, label='Actual')
plt.plot(test_data['date'], forecast_xgb, color='red', label='XGBoost Forecast')
plt.legend()
plt.title('XGBoost Forecast vs Actuals')
plt.show()

mse_xgb = mean_squared_error(y_test, forecast_xgb)
rmse_xgb = np.sqrt(mse_xgb)
rmsle_xgb = rmsle(forecast_xgb, y_test)

In [ ]:
X_train_values = X_train[exog_features].values.reshape(X_train.shape[0], 1, len(exog_features))
X_test_values = X_test[exog_features].values.reshape(X_test.shape[0], 1, len(exog_features))

# LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(50, input_shape=(X_train_values.shape[1], X_train_values.shape[2])))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mse', optimizer='adam')

model_lstm.fit(X_train_values, y_train, epochs=50, batch_size=72, validation_data=(X_test_values, y_test), verbose=2, shuffle=False)

forecast_lstm = model_lstm.predict(X_test_values).flatten()

plt.figure(figsize=(10,6))
plt.plot(test_data['date'], y_test, label='Actual')
plt.plot(test_data['date'], forecast_lstm, color='red', label='LSTM Forecast')
plt.legend()
plt.title('LSTM Forecast vs Actuals')
plt.show()

mse_lstm = mean_squared_error(y_test, forecast_lstm)
rmse_lstm = np.sqrt(mse_lstm)
rmsle_lstm = rmsle(forecast_lstm, y_test)

In [ ]:
# Create a dictionary of RMSE and MSE for each model
data = {
    'Model': ['ARIMA', 'SARIMA', 'ExponentialSmoothing', 'XGBoost', 'LSTM'],
    'RMSE': [rmse_arima, rmse_sarima, rmse_ets, rmse_xgb, rmse_lstm],
    'MSE': [mse_arima, mse_sarima, mse_ets, mse_xgb, mse_lstm],
    'RMSLE': [rmsle_arima, rmsle_sarima, rmsle_ets, rmsle_xgb, rmsle_lstm]
}

range_of_y_test = y_test.max() - y_test.min()

# Convert dictionary to DataFrame
score_df = pd.DataFrame(data)

percentage_rmse = score_df['RMSE'] * 100
percentage_mse = score_df['MSE'] * 100

score_df['RMSE (%)'] = percentage_rmse
score_df['MSE (%)'] = percentage_mse

# Display the scores table
print(score_df) 


<h1 style='background:#0A4D68; border:3; color:cyan; border-color:cyan; border-style:dotted;'><center> ADVANCED MODEL EVALUATION</center></h1> 

In [ ]:
# TIme series splitting for cross Validation
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
# Cross Validating ARIMA Model
mse_scores_arima = []
rmsle_scores_arima = []  # List to collect RMSLE scores

for train_index, test_index in tscv.split(y_train):
    train, test = y_train.iloc[train_index], y_train.iloc[test_index]
    exog_train, exog_test = X_train.iloc[train_index], X_train.iloc[test_index]
    
    model_arima = sm.tsa.SARIMAX(train, exog=exog_train[exog_features], order=(1,0,0))
    results_arima = model_arima.fit(disp=False)
    forecast_arima = results_arima.predict(start=len(train), end=len(train) + len(test) - 1, exog=exog_test[exog_features])
    
    mse_arima = mean_squared_error(test, forecast_arima)
    mse_scores_arima.append(mse_arima)

    # Computing RMSLE using your custom function
    rmsle_value = rmsle(forecast_arima, test)
    rmsle_scores_arima.append(rmsle_value)

In [ ]:
mse_scores_sarima = []
rmsle_scores_sarima = []  # List to collect RMSLE scores

for train_index, test_index in tscv.split(y_train):
    train, test = y_train.iloc[train_index], y_train.iloc[test_index]
    exog_train, exog_test = X_train.iloc[train_index], X_train.iloc[test_index]
    
    model_sarima = sm.tsa.SARIMAX(train, exog=exog_train[exog_features], order=(1,1,0), seasonal_order=(1,0,0,12))
    results_sarima = model_sarima.fit(disp=False)
    forecast_sarima = results_sarima.predict(start=len(train), end=len(train) + len(test) - 1, exog=exog_test[exog_features])
    
    mse_sarima = mean_squared_error(test, forecast_sarima)
    mse_scores_sarima.append(mse_sarima)

    # Computing RMSLE using your custom function
    rmsle_value = rmsle(forecast_sarima, test)
    rmsle_scores_sarima.append(rmsle_value)

In [ ]:
mse_scores_ets = []
rmsle_scores_ets = []  # List to collect RMSLE scores

for train_index, test_index in tscv.split(y_train):
    train, test = y_train.iloc[train_index], y_train.iloc[test_index]
    exog_train, exog_test = X_train.iloc[train_index], X_train.iloc[test_index]
    
    model_ets = ExponentialSmoothing(train, seasonal='add', seasonal_periods=12)
    results_ets = model_ets.fit()
    initial_forecast = results_ets.forecast(steps=len(test))

    X_adjust_train = pd.concat([train.shift(-1).dropna().rename("forecast"), exog_train[exog_features].iloc[1:, :]], axis=1)
    X_adjust_test = pd.concat([pd.Series(initial_forecast, index=test.index).rename("forecast"), exog_test[exog_features]], axis=1)
    
    # Remove rows with NaN values
    X_adjust_train = X_adjust_train.dropna()
    y_adjust_train = train.loc[X_adjust_train.index]

    X_adjust_test = X_adjust_test.dropna()
    y_adjust_test = test.loc[X_adjust_test.index]

    # Convert all feature names to strings
    X_adjust_train.columns = X_adjust_train.columns.astype(str)
    X_adjust_test.columns = X_adjust_test.columns.astype(str)

    # Fit the Linear Regression Model
    lr = LinearRegression()
    lr.fit(X_adjust_train, y_adjust_train)
    forecast_ets = lr.predict(X_adjust_test)
    
    mse_ets = mean_squared_error(y_adjust_test, forecast_ets)
    mse_scores_ets.append(mse_ets)

    # Computing RMSLE using your custom function
    rmsle_value = rmsle(forecast_ets, y_adjust_test)
    rmsle_scores_ets.append(rmsle_value)

In [ ]:
mse_scores_xgb = []
rmsle_scores_xgb = []  # List to collect RMSLE scores

for train_index, test_index in tscv.split(y_train):
    train, test = y_train.iloc[train_index], y_train.iloc[test_index]
    exog_train, exog_test = X_train.iloc[train_index], X_train.iloc[test_index]
    
    model_xgb = xgb.XGBRegressor(objective ='reg:squarederror')
    model_xgb.fit(exog_train, train)
    forecast_xgb = model_xgb.predict(exog_test)
    
    mse_xgb = mean_squared_error(test, forecast_xgb)
    mse_scores_xgb.append(mse_xgb)

    # Computing RMSLE using your custom function
    rmsle_value = rmsle(forecast_xgb, test)
    rmsle_scores_xgb.append(rmsle_value)

In [ ]:
#Cross Validation of LSTM Model 
rmsle_scores = []

for train_index, test_index in tscv.split(X_train_values):
    
    X_train_cv, X_test_cv = X_train_values[train_index], X_train_values[test_index]
    y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]
    
    model_lstm = Sequential()
    model_lstm.add(LSTM(50, input_shape=(X_train_cv.shape[1], X_train_cv.shape[2])))
    model_lstm.add(Dense(1))
    model_lstm.compile(loss='mse', optimizer='adam')
    model_lstm.fit(X_train_cv, y_train_cv, epochs=50, batch_size=72, validation_data=(X_test_cv, y_test_cv), verbose=2, shuffle=False)
    
    forecast_lstm = model_lstm.predict(X_test_cv).flatten()
    score = rmsle(forecast_lstm, y_test_cv)
    rmsle_scores.append(score)
    print(f'RMSLE Score for split {len(rmsle_scores)}: {score}')

print(f'Average RMSLE Score: {np.mean(rmsle_scores)}')

In [ ]:
# Create a DataFrame for RMSLE
df_rmsle_scores = pd.DataFrame({
    'ARIMA': rmsle_scores_arima,
    'SARIMA': rmsle_scores_sarima,
    'ETS': rmsle_scores_ets,
    'XGBoost': rmsle_scores_xgb,
    'LSTM' : rmsle_scores  
})

# Compute the average RMSLE across all folds for each model
avg_rmsle = df_rmsle_scores.mean().sort_values(ascending=True)  # Lower RMSLE values are better, hence sort in ascending

print(avg_rmsle)

XGBoost: With an RMSLE of 0.007496, this model demonstrates the best performance among all the models tested. It has the closest predictions to the actual data, with the error being the smallest.

LSTM: The LSTM model, a type of recurrent neural network, yielded an RMSLE of 0.034217. While it performs reasonably well, it's not as precise as the XGBoost model.

ETS: The Exponential Smoothing model has an RMSLE of 0.035308, which is close to the LSTM model's performance but slightly worse. ETS is a time series forecasting method that captures different time series components like trend and seasonality.

ARIMA: The AutoRegressive Integrated Moving Average model produced an RMSLE of 0.051475. ARIMA is a classical method in time series forecasting that combines autoregressive, differencing, and moving average components.

SARIMA: The Seasonal ARIMA, which extends ARIMA by adding a seasonal component, has the highest RMSLE of 0.054316. This suggests it's the least accurate among the models tested on this dataset.

The SARIMA model was dropped from consideration due to its high RMSLE score of 0.054316. This score indicates that, among the models tested, SARIMA had the least accurate predictions, making other models, especially XGBoost with its significantly lower RMSLE, preferable choices for this forecasting task.



<h1 style='background:#0A4D68; border:3; color:cyan; border-color:cyan; border-style:dotted;'><center> HYPERPARAMETER TUNING</center></h1> 

In [ ]:
# For ARIMA
auto_arima_model = pm.auto_arima(y_train, exog=X_train[exog_features], trace=True, error_action='ignore', suppress_warnings=True)
print(auto_arima_model.summary())

In [ ]:
# Modified ETS hyperparameters to try
ets_param_grid = {
    'trend': ['add', None],
    'seasonal': ['add', None],
    'seasonal_periods': list(range(7, 365, 12))  
}
best_ets_params = None
best_ets_mse = float("inf")

# ETS hyperparameter tuning
for param in ParameterGrid(ets_param_grid):
    try:
        model_ets = ExponentialSmoothing(y_train, **param)
        results_ets = model_ets.fit()
        forecast_ets = results_ets.forecast(steps=len(y_test))
        mse_ets = mean_squared_error(y_test, forecast_ets)
        if mse_ets < best_ets_mse:
            best_ets_mse = mse_ets
            best_ets_params = param
    except:
        continue

print("Best ETS parameters:", best_ets_params)

In [ ]:

# XGBoost hyperparameters to try
xgb_param_grid = {
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 4, 6]
}

model_xgb = xgb.XGBRegressor(objective='reg:squarederror')
grid_search = GridSearchCV(model_xgb, xgb_param_grid, cv=10, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_xgb_params = grid_search.best_params_

print("Best XGBoost parameters:", best_xgb_params)



<h1 style='background:#0A4D68; border:3; color:cyan; border-color:cyan; border-style:dotted;'><center> FITTING TUNED MODELS</center></h1> 

In [ ]:
# Using best parameters for ARIMA
model_arima = sm.tsa.SARIMAX(y_train, 
                            exog=X_train[exog_features], 
                            order=(5, 1, 2))  # Updated order parameters
results_arima = model_arima.fit(disp=False)
forecast_arima = results_arima.predict(start=len(y_train), 
                                       end=len(y_train) + len(y_test) - 1, 
                                       exog=X_test[exog_features])

# Visualization
plt.figure(figsize=(10,6))
plt.plot(test_data['date'], y_test, label='Actual')
plt.plot(test_data['date'], forecast_arima, color='red', label='ARIMA Forecast')
plt.legend()
plt.title('ARIMA Forecast vs Actuals')
plt.show()

# Performance metrics
mse_arima = mean_squared_error(y_test, forecast_arima)
rmse_arima = np.sqrt(mse_arima)
rmsle_arima_best = rmsle(forecast_arima, y_test)


print("Mean Squared Error:", mse_arima)
print("RMSLE", rmsle_arima)


In [ ]:

# 1. Fit ETS Model and Make Initial Forecasts using Best Parameters
model_ets = ExponentialSmoothing(y_train, seasonal='add', seasonal_periods=55, trend='add')
results_ets = model_ets.fit()
initial_forecast = results_ets.forecast(steps=len(y_test))

# 2. Prepare Adjusted Exogenous Data for Linear Regression

# Using the initial forecast and exog features to predict y_test.
X_adjust_train = pd.concat([y_train.shift(-1).dropna().rename("forecast"), X_train[exog_features].iloc[1:, :]], axis=1)
X_adjust_test = pd.concat([pd.Series(initial_forecast, index=y_test.index).rename("forecast"), X_test[exog_features]], axis=1)

# Remove rows with NaN values
X_adjust_train = X_adjust_train.dropna()
y_adjust_train = y_train.loc[X_adjust_train.index]

X_adjust_test = X_adjust_test.dropna()
y_adjust_test = y_test.loc[X_adjust_test.index]

# Convert all feature names to strings
X_adjust_train.columns = X_adjust_train.columns.astype(str)
X_adjust_test.columns = X_adjust_test.columns.astype(str)

# 3. Fit the Linear Regression Model

lr = LinearRegression()
lr.fit(X_adjust_train, y_adjust_train)
forecast_ets = lr.predict(X_adjust_test)

# 4. Visualization
plt.figure(figsize=(10,6))
plt.plot(y_adjust_test.index, y_adjust_test, label='Actual')
plt.plot(y_adjust_test.index, forecast_ets, color='red', label='Adjusted ETS Forecast')
plt.legend()
plt.title('Adjusted ETS Forecast vs Actuals')
plt.show()

# 5. Performance Metrics
mse_ets = mean_squared_error(y_adjust_test, forecast_ets)
rmse_ets = np.sqrt(mse_ets)
rmsle_ets_best = rmsle(forecast_ets, y_adjust_test)


print("RMSLE", rmsle_ets_best)


In [ ]:
# Using the best parameters to fit the XGBoost model
best_params_xgb = {'learning_rate': 0.3, 'max_depth': 6, 'n_estimators': 200}

model_xgb_best = xgb.XGBRegressor(**best_params_xgb, objective='reg:squarederror')
model_xgb_best.fit(X_train, y_train)
forecast_xgb_best = model_xgb_best.predict(X_test)


plt.figure(figsize=(10,6))
plt.plot(test_data['date'], y_test, label='Actual')
plt.plot(test_data['date'], forecast_xgb_best, color='red', label='XGBoost Forecast')
plt.legend()
plt.title('XGBoost Forecast vs Actuals')
plt.show()


# Calculate RMSE
mse_xgb_best = mean_squared_error(y_test, forecast_xgb_best)
rmse_xgb_best = np.sqrt(mse_xgb_best)
rmsle_xgb_best = rmsle(forecast_xgb_best, y_test)



print("RMSlE with best XGBoost parameters:", rmsle_xgb_best)


After an extensive hyperparameter tuning process, we identified the optimal settings for our XGBoost model. The best model, equipped with these fine-tuned parameters, achieved an impressive Root Mean Squared Logarithmic Error (RMSLE) of 0.0053827804583248175. This low RMSLE score underscores the model's accuracy and the efficacy of the tuning process in optimizing its performance and predicting sales.



<h1 style='background:#0A4D68; border:3; color:cyan; border-color:cyan; border-style:dotted;'><center> TEST PREDICTIONS</center></h1> 

In [ ]:
# Aggregating data
df_test_aggregated = df_test.groupby(['date', 'family']).agg({
    'onpromotion': 'sum'    
}).reset_index()

In [ ]:
df_test_aggregated.shape

In [ ]:
# 1. Day of the week
df_test_aggregated['day_of_week'] = df_test_aggregated['date'].dt.dayofweek

In [ ]:
# 2. Lagged variables
# Since this is test data and we do not have prior data to create lag, 
# we can set lagged values to 0 
df_test_aggregated['lag_1'] = 0

In [ ]:
# 3. Rolling statistics
# Same as lagged variables, since this is test data, we can't compute rolling mean.
# We can set it to 0 
df_test_aggregated['rolling_mean'] = 0

In [ ]:
# One-hot encoding for family column
df_test_dummies = pd.get_dummies(df_test_aggregated['family'], drop_first=True, prefix='family')
df_test_aggregated = pd.concat([df_test_aggregated, df_test_dummies], axis=1)
df_test_aggregated.drop('family', axis=1, inplace=True)

In [ ]:
# Create a new scaler without the 'sales' column
columns_to_scale_without_sales = ['onpromotion', 'day_of_week', 'lag_1', 'rolling_mean'] + list(df_family_dummies.columns)
scaler_without_sales = MinMaxScaler()
scaler_without_sales.fit(df_family_aggregated[columns_to_scale_without_sales])

# Now, use this new scaler to transform the test set
df_test_aggregated[columns_to_scale_without_sales] = scaler_without_sales.transform(df_test_aggregated[columns_to_scale_without_sales])

df_test_aggregated = df_test_aggregated.sort_values('date')

In [ ]:
# Making sure to only consider the features the model was trained on.
features_considered = X_train.columns.tolist()
X_new_test = df_test_aggregated[features_considered]

# Predicting with the model
forecast_xgb_new_test = model_xgb_best.predict(X_new_test)

# Combine the forecasts with the aggregated dataset
df_test_aggregated['forecast'] = forecast_xgb_new_test

# Group by the dates and take the sum of the forecasts
datewise_forecast = df_test_aggregated.groupby('date')['forecast'].sum()

# Now, plot these aggregated forecasts
plt.figure(figsize=(10,6))
plt.plot(datewise_forecast.index, datewise_forecast.values, color='red', label='XGBoost Forecast for new test data')
plt.legend()
plt.title('XGBoost Forecast for new test data')
plt.show()

In [ ]:
# Display the head of X_train
print("X_train:")
print(X_train.head())

# Display the head of y_train
print("\ny_train:")
print(y_train.head())


BELOW WE ARE SAVING OUR MODELS 

In [ ]:
# Saving our best trained XGBoost model to a file
model_filename = "xgboost_model.pkl"
joblib.dump(model_xgb_best, model_filename)

In [144]:
# saving our preprocessors as files
joblib.dump(scaler, 'standard_scaler.pkl')

['standard_scaler.pkl']

In [146]:
# saving the one-hot encoding
encoding_file_path = 'family_encoding.pkl'
# Serialize and save the one-hot encoded DataFrame
with open(encoding_file_path, 'wb') as file:
    pickle.dump(df_family_aggregated, file)